In [ ]:
import pandas as pd
from dash import Dash, html, dcc
from dash.dependencies import Output, Input
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
import base64
from io import BytesIO

# Load and preprocess the data
file_path = 'mass_shootings_with_lat_and_long.csv'
df = pd.read_csv(file_path)
df['Incident Date'] = pd.to_datetime(df['Incident Date'], dayfirst=True)  # Convert to datetime
df['year'] = df['Incident Date'].dt.year  # Extract year
df['Total Victims'] = df['Killed'] + df['Injured']  # Calculate total victims (killed + injured)


# Initialize Dash app
app = Dash(__name__)

#Timeseries
def draw_time_series(df, selected_variable):
    grouped_df = df.groupby(df['Incident Date'].dt.to_period('M')).agg({selected_variable: 'sum'}).reset_index()
    grouped_df['Incident Date'] = grouped_df['Incident Date'].astype(str)
    fig = px.line(
        grouped_df, x='Incident Date', y=selected_variable, 
        title=f'Sum of {selected_variable.capitalize()} Over Time',
        labels={'Incident Date': 'Date', selected_variable: f'Sum of {selected_variable.capitalize()}'}
    )
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        title={
            'text': f'Sum of {selected_variable.capitalize()} Over Time',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title='Date',
        yaxis_title=f'Sum of {selected_variable.capitalize()}',
        font=dict(
            color='black'
        )
    )
    fig.update_traces(line=dict(color='red'))
    return fig
    
# Map figure
def draw_map(map_df):
    fig = go.Figure(go.Scattermapbox(
        mode='markers',
        lon=map_df['Longitude'],
        lat=map_df['Latitude'],
        marker=go.scattermapbox.Marker(
            size=map_df['Total Victims'],
            sizemode='area',
            sizemin=5,
            sizeref=2.*max(map_df['Total Victims'])/(50.**2),
            color=map_df['Total Victims'],
            colorscale='Reds',
            showscale=True
        ),
        text=map_df.apply(lambda row: f"{row['City Or County']}<br>"
                                      f"Date: {row['Incident Date'].date()}<br>"
                                      f"Victims: {row['Total Victims']}", axis=1),
        hoverinfo='text'
    ))

    fig.update_layout(
        mapbox_style='open-street-map',
        mapbox_zoom=3,
        mapbox_center={'lat': 37.0902, 'lon': -95.7129},  # Centered on the US
        height=600,
        margin={'r': 0, 't': 0, 'l': 0, 'b': 0}
    )

    return fig
#Wordcloud
def generate_wordcloud(data):
    wc = WordCloud(width=1000, height=400, background_color='white', colormap='Reds').generate_from_frequencies(data)
    img = BytesIO()
    wc.to_image().save(img, format='PNG')
    img.seek(0)
    return base64.b64encode(img.getvalue()).decode()

# Default figures
default_variable = 'Total Victims'
default_map_fig = draw_map(df)
default_time_series_fig = draw_time_series(df, default_variable)

# Layout
app.layout = html.Div([
    html.H1('Mass Shootings in US', style={'textAlign': 'center', 'margin-bottom': '20px'}),
    dcc.Dropdown(
        ['Total Victims', 'Killed', 'Injured'],  # Options for dropdown
        'Total Victims',  # Default value
        id='variable_dropdown'
    ),
    dcc.Dropdown(
        options=[{'label': str(year), 'value': year} for year in sorted(df['year'].unique())] + [{'label': 'All Years', 'value': 'All Years'}],
        value='All Years',  # Default value
        id='year_dropdown'
    ),
    dcc.Graph(id='time_series_graph', figure=default_time_series_fig),
    dcc.Graph(id='map_graph', figure=default_map_fig),
    html.Div(id='incident_count', style={'textAlign': 'center', 'fontSize': 18, 'margin-top': '20px'}),
    html.Label('Victim amount'),
    dcc.RangeSlider(
        min=df[default_variable].min(),
        max=df[default_variable].max(),
        value=[df[default_variable].min(), df[default_variable].max()],
        marks={str(i): str(i) for i in range(0, int(df[default_variable].max()) + 1, 10)},
        step=1,
        id='incident_slider'
    ),
    html.Div([
        html.Label('Incidents by State', style={'textAlign': 'center', 'fontSize': 18, 'margin-top': '20px'}),
        html.Img(id='wordcloud', style={'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
    ])
])

# Callback to update map and incident count based on selected year
@app.callback(
    [Output('map_graph', 'figure'), Output('incident_count', 'children')],
    [Input('year_dropdown', 'value'),Input('incident_slider','value')]
)
def update_map_and_count(selected_year,incident_range):
    filtered_df = df.copy()
    if selected_year != 'All Years':
        filtered_df = filtered_df[filtered_df['year'] == int(selected_year)]
    
    # Filter by incident range (slider)
    filtered_df = filtered_df[
        (filtered_df['Total Victims'] >= incident_range[0]) & 
        (filtered_df['Total Victims'] <= incident_range[1])
    ]

    map_fig = draw_map(filtered_df)
    
    incident_count = f'Total Incidents in Selected Year: {len(filtered_df["Incident ID"].unique())}'

    return map_fig, incident_count

# Callback to update word cloud based on selected year, variable, and sliders
@app.callback(
    Output('wordcloud', 'src'),
    [Input('year_dropdown', 'value'), Input('variable_dropdown', 'value'), Input('incident_slider', 'value')]
)
def update_wordcloud(selected_year, selected_variable, incident_range):
    filtered_df = df.copy()
    if selected_year != 'All Years':
        filtered_df = filtered_df[filtered_df['year'] == int(selected_year)]

    filtered_df = filtered_df[(filtered_df[selected_variable] >= incident_range[0]) & (filtered_df[selected_variable] <= incident_range[1])]
    
    # Generate word cloud image
    state_counts = filtered_df['State'].value_counts()
    wordcloud_src = f"data:image/png;base64,{generate_wordcloud(state_counts)}"
    
    return wordcloud_src

# Callback to update time series based on selected year and variable
@app.callback(
    Output('time_series_graph', 'figure'),
    [Input('year_dropdown', 'value'), Input('variable_dropdown', 'value')]
)
def update_time_series(selected_year, selected_variable):
    filtered_df = df.copy()
    if selected_year != 'All Years':
        filtered_df = filtered_df[filtered_df['year'] == int(selected_year)]
    
    time_series_fig = draw_time_series(filtered_df, selected_variable)
    return time_series_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8054)
